# With rasterio `rasterize`

Rasterix includes a dask-aware wrapper for `rasterio.features.rasterize`.

## Read data

### Read in some raster data

In [ ]:
import xarray as xr
import xproj  # noqa

ds = xr.tutorial.open_dataset("eraint_uvz")
ds = ds.proj.assign_crs(spatial_ref="epsg:4326")
ds

### Read in example geometries

In [ ]:
import geodatasets
import geopandas as gpd

world = gpd.read_file(geodatasets.get_path("naturalearth land"))
world

In [ ]:
from rasterix.rasterize import rasterize

n = rasterize(ds, world[["geometry"]], xdim="longitude", ydim="latitude", engine="rasterio")
n.plot()

## Out-of-core support

All combinations of chunked and in-memory arrays and geometries are supported.

### dask.array + geopandas

In [ ]:
chunked = ds.chunk({"latitude": -1, "longitude": 120})
d = rasterize(chunked, world[["geometry"]], xdim="longitude", ydim="latitude", engine="rasterio")
d

In [ ]:
xr.testing.assert_identical(n, d)

### dask.array + dask-geopandas

In [ ]:
import dask_geopandas as dgpd

dd = rasterize(
    ds.chunk({"latitude": -1, "longitude": 240}),
    dgpd.from_geopandas(world[["geometry"]], npartitions=3),
    xdim="longitude",
    ydim="latitude",
    engine="rasterio",
)
dd

In [ ]:
xr.testing.assert_identical(dd, d)